<a href="https://colab.research.google.com/github/snaiws/Project_5/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#라이브러리
import requests
from urllib import parse

In [5]:
#버전,티어별 닉넴 뽑기
def getSummonerid(tier,page,KEY): # II구간 get
  APIURL = f"https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/II?page={page}&api_key={KEY}"
  res = requests.get(APIURL)
  data = res.json()
  data = [x['summonerName'] for x in data if x['inactive'] == False]
  return data
KEY = 'RGAPI-e614c53d-6964-4275-af3e-dc6907cce862'
a=getSummonerid('DIAMOND','1',KEY)
b=getSummonerid('DIAMOND','2',KEY)
print(len(a))
print(len(b))
print(len(list(set(a+b))))

205
205
405


In [12]:
#get puuid
def getPuuid(summonerName,KEY):
  encodedSummonerName = parse.quote(summonerName)
  APIURL = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{encodedSummonerName}?api_key={KEY}"
  res = requests.get(APIURL)
  data = res.json()
  return data['puuid']

In [13]:
getPuuid('지 감',KEY)

'5WhmVKS1x5F_yuGm_aAw9FPjZOLZuCIrCF-QFPjoNyhYi7u00j9JElyqdRaVxfRFUzS4rJAZMNpRDA'

In [17]:
#get MatchId
def getMatchId(puuid,KEY):
  APIURL = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key={KEY}"
  res = requests.get(APIURL)
  data = res.json()
  return data

In [18]:
getMatchId('5WhmVKS1x5F_yuGm_aAw9FPjZOLZuCIrCF-QFPjoNyhYi7u00j9JElyqdRaVxfRFUzS4rJAZMNpRDA',KEY)

['KR_5543645759',
 'KR_5543615983',
 'KR_5543652588',
 'KR_5543610885',
 'KR_5543575801',
 'KR_5543533367',
 'KR_5543560422',
 'KR_5543164873',
 'KR_5543120167',
 'KR_5542271662',
 'KR_5542231434',
 'KR_5542230306',
 'KR_5542215801',
 'KR_5542144939',
 'KR_5527024871',
 'KR_5526943325',
 'KR_5526918115',
 'KR_5526894273',
 'KR_5526836874',
 'KR_5526803408']

In [22]:
#getMatch
def getMatch(matchId,KEY):
  APIURL = f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={KEY}"
  res = requests.get(APIURL)
  data = res.json()
  return data

In [24]:
a=getMatch('KR_5517613909',KEY)

In [25]:
a

{'info': {'gameCreation': 1634544663000,
  'gameDuration': 1555,
  'gameEndTimestamp': 1634546247911,
  'gameId': 5517613909,
  'gameMode': 'CLASSIC',
  'gameName': 'teambuilder-match-5517613909',
  'gameStartTimestamp': 1634544692752,
  'gameType': 'MATCHED_GAME',
  'gameVersion': '11.20.400.7328',
  'mapId': 11,
  'participants': [{'assists': 7,
    'baronKills': 0,
    'bountyLevel': 0,
    'champExperience': 15306,
    'champLevel': 16,
    'championId': 150,
    'championName': 'Gnar',
    'championTransform': 0,
    'consumablesPurchased': 3,
    'damageDealtToBuildings': 7686,
    'damageDealtToObjectives': 7686,
    'damageDealtToTurrets': 7686,
    'damageSelfMitigated': 23646,
    'deaths': 9,
    'detectorWardsPlaced': 0,
    'doubleKills': 0,
    'dragonKills': 0,
    'firstBloodAssist': False,
    'firstBloodKill': False,
    'firstTowerAssist': False,
    'firstTowerKill': True,
    'gameEndedInEarlySurrender': False,
    'gameEndedInSurrender': False,
    'goldEarned': 1